# Image Stitching

In [1]:
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt

## Code

In [4]:
# 主代码部分
img1 = cv2.imread('scenery1.jpg')
img2 = cv2.imread('scenery2.jpg')

sift = cv2.xfeatures2d.SIFT_create()
    
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)
    
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1, des2, k = 2) 
    
#     print('kp1:',kp1[0])            kp1: <KeyPoint 000001918B2CC5D0>                                      
#     print('des1:',des1.shape)       des1: (3967, 128)                                                    
#     print('matches:',matches[0])    matches: [<DMatch 000001918B698510>, <DMatch 000001918B698A10>] 
    
#     print(type(kp1[0]))             <class 'cv2.KeyPoint'> 
#     print(type(des1))               <class 'numpy.ndarray'>
#     print(type(matches[0][0]))      <class 'cv2.DMatch'>

good = [[m] for m, n in matches if m.distance < 0.5 * n.distance]   # ratio等于0.5


src_pts = np.array([kp1[m.queryIdx].pt for [m] in good])    # 原图的点坐标 img1
dst_pts = np.array([kp2[m.trainIdx].pt for [m] in good])    # 目标图的点坐标 img2

H = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)         # 生成单应性矩阵

h, w = img1.shape[: 2]
h1, w1 = img2.shape[: 2]

shift = np.array([[1.0, 0, w], [0, 1.0, 0], [0, 0, 1.0]])
M = np.dot(shift, H[0])            # 获取左边图像到右边图像的投影映射关系

dst_corners = cv2.warpPerspective(img1, M, (w * 2, h))# 透视变换，新图像可容纳完整的两幅图

dst_corners[0: h1, w: w + w1] = img2  # 将第二幅图放在右侧

In [5]:
cv2.imshow('Stitched_img',dst_corners)

key = cv2.waitKey(0)
if key == 27:
    cv2.destroyAllWindows()

## Reference

- [关于knnMatch](https://www.cnblogs.com/zyly/p/9646201.html)
- [HOG](https://blog.csdn.net/passball/article/details/82254256)
- [SIFT详解](https://www.jianshu.com/p/14b92d3fd6f8)